In [ ]:
# Copyright (c) 2025 science42.tech, science42 Authors. All Rights Reserved.
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.
#
# Author: Zhicheng Wang, Kang Ming, Hui Xiang
import scipy.io 
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

import torch
from lib.PINN.Fluid_Mechanics.RANS_backstep.tools import *
from lib.PINN.Fluid_Mechanics.RANS_backstep import rans_data as rdata
from lib.PINN.Fluid_Mechanics.RANS_backstep import pinn_solver_2D as psolver


def test(net_params=None, net_params_1=None, loop=0):
    nu_0 = 1e-5   # 动力粘度
    U_0 = 1.0     # 入口速度
    rho_0 = 1.0   # 密度
    L_0 = 1.0     # 特征长度
    N_neu = 120   # 神经网络隐藏层神经元数量
    N_neu_1 = 40  # 神经网络隐藏层神经元数量（第二个网络）
    lam_bcs = 10  # 边界条件权重
    lam_equ = 1   # 方程权重
    N_f = 40000   # 训练点数量
    alpha_evm = 0.03  # EVM参数
    beta_evm = 10.0   # EVM参数
    N_HLayer = 4      # 隐藏层数量
    N_HLayer_1 = 4    # 隐藏层数量（第二个网络）
   
    PINN = psolver.PhysicsInformedNeuralNetwork(
        nu_0=nu_0,
        U_0=U_0,
        rho_0=rho_0,
        L_0=L_0,
        layers=N_HLayer,
        layers_1=N_HLayer_1,
        hidden_size=N_neu,
        hidden_size_1=N_neu_1,
        N_f=N_f,
        alpha_evm=alpha_evm,
        beta_evm=beta_evm,
        bc_weight=lam_bcs,
        eq_weight=lam_equ,
        net_params=net_params,
        checkpoint_path='./checkpoint/')
    
    dataloader = rdata.DataLoader(path='./datasets/', N_f=N_f, N_b=1000)
    ref_file = '/data/datasets/RANS_backstep/Re1e5_backStep_L8.mat'
    x_star, y_star, u_star, v_star, p_star, k_star, o_star, n_star = dataloader.loading_evaluate_data(ref_file)
    PINN.set_testing_data(x_star, y_star, u_star, v_star, p_star, k_star, o_star)
    res_file = PINN.predict(x_star, y_star, u_star, v_star)
    plot_images(ref_file, res_file)

def plot_images(ref_file, res_file):
    ref_file = ref_file
    reference = scipy.io.loadmat(ref_file)
    X_ref = reference['X_ref']
    Y_ref = reference['Y_ref']
    u_ref = reference['U_ref']
    v_ref = reference['V_ref']
    p_ref = reference['P_ref']
    p_ref = p_ref - np.nanmean(p_ref)
    k_ref = reference['K_ref']
    o_ref = reference['O_ref']

    Z_ref = np.zeros_like(Y_ref)

    # 加载预测结果文件
    result_file = res_file
    RESULT = scipy.io.loadmat(result_file)
    
    # 转换为双精度浮点数
    u_pred = RESULT['U_pred'].astype(np.float64)
    v_pred = RESULT['V_pred'].astype(np.float64)
    p_pred = RESULT['P_pred'].astype(np.float64)

    # 调整预测压力，减去平均值
    p_pred = p_pred - np.mean(p_pred)

    # 计算误差
    u_err = np.flipud(np.abs(u_pred - u_ref))
    v_err = np.flipud(np.abs(v_pred - v_ref))
    p_err = np.flipud(np.abs(p_pred - p_ref))

    # 计算L2误差
    L2_u = np.linalg.norm(u_pred - u_ref) / np.linalg.norm(u_ref)
    L2_v = np.linalg.norm(v_pred - v_ref) / np.linalg.norm(v_ref)
    L2_p = np.linalg.norm(p_pred[~np.isnan(p_ref)] - p_ref[~np.isnan(p_ref)]) / np.linalg.norm(p_ref[~np.isnan(p_ref)])

    print('L2 error :')
    print(f'----- L2_u = {L2_u * 100:.3f} %')
    print(f'----- L2_v = {L2_v * 100:.3f} %')
    print(f'----- L2_p = {L2_p * 100:.3f} %')

    # 定义网格以进行插值
    x1 = np.linspace(-4.0, 4.0, 257)
    y1 = np.linspace(0.0, 8.0, 257)
    X_ref1, Y_ref1 = np.meshgrid(x1, y1)

    # 网格插值
    U_ref1 = griddata((X_ref.ravel(), Y_ref.ravel()), u_ref.ravel(), (X_ref1, Y_ref1), method='linear')
    V_ref1 = griddata((X_ref.ravel(), Y_ref.ravel()), v_ref.ravel(), (X_ref1, Y_ref1), method='linear')
    P_ref1 = griddata((X_ref.ravel(), Y_ref.ravel()), p_ref.ravel(), (X_ref1, Y_ref1), method='linear')
    
    U_pred1 = griddata((X_ref.ravel(), Y_ref.ravel()), u_pred.ravel(), (X_ref1, Y_ref1), method='linear')
    V_pred1 = griddata((X_ref.ravel(), Y_ref.ravel()), v_pred.ravel(), (X_ref1, Y_ref1), method='linear')
    P_pred1 = griddata((X_ref.ravel(), Y_ref.ravel()), p_pred.ravel(), (X_ref1, Y_ref1), method='linear')

    U_err1 = griddata((X_ref.ravel(), Y_ref.ravel()), u_err.ravel(), (X_ref1, Y_ref1), method='linear')
    V_err1 = griddata((X_ref.ravel(), Y_ref.ravel()), v_err.ravel(), (X_ref1, Y_ref1), method='linear')
    P_err1 = griddata((X_ref.ravel(), Y_ref.ravel()), p_err.ravel(), (X_ref1, Y_ref1), method='linear')


    # 边界条件调整
    for it1 in range(len(x1)):
        for it2 in range(len(y1)):
            if x1[it1] < 0 and y1[it2] < 1.0:
                U_ref1[it2, it1] = np.nan
                V_ref1[it2, it1] = np.nan
                P_ref1[it2, it1] = np.nan
                U_pred1[it2, it1] = np.nan
                V_pred1[it2, it1] = np.nan
                P_pred1[it2, it1] = np.nan
                U_err1[it2, it1] = np.nan
                V_err1[it2, it1] = np.nan
                P_err1[it2, it1] = np.nan

    # 绘图
    fig, axs = plt.subplots(3, 3, figsize=(15, 15))
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    # U速度图
    axs[0, 0].contourf(X_ref1, Y_ref1, U_ref1, 50, cmap='jet')
    axs[0, 0].set_title('u_{ref}')
    axs[0, 1].contourf(X_ref1, Y_ref1, U_pred1, 50, cmap='jet')
    axs[0, 1].set_title('u_{NN}')
    axs[0, 2].contourf(X_ref1, Y_ref1, U_err1, 50, cmap='jet')
    axs[0, 2].set_title('|u_{ref} - u_{NN}|')

    # V速度图
    axs[1, 0].contourf(X_ref1, Y_ref1, V_ref1, 50, cmap='jet')
    axs[1, 0].set_title('v_{ref}')
    axs[1, 1].contourf(X_ref1, Y_ref1, V_pred1, 50, cmap='jet')
    axs[1, 1].set_title('v_{NN}')
    axs[1, 2].contourf(X_ref1, Y_ref1, V_err1, 50, cmap='jet')
    axs[1, 2].set_title('|v_{ref} - v_{NN}|')

    # 压力图
    axs[2, 0].contourf(X_ref1, Y_ref1, P_ref1, 50, cmap='jet')
    axs[2, 0].set_title('p_{ref}')
    axs[2, 1].contourf(X_ref1, Y_ref1, P_pred1, 50, cmap='jet')
    axs[2, 1].set_title('p_{NN}')
    axs[2, 2].contourf(X_ref1, Y_ref1, P_err1, 50, cmap='jet')
    axs[2, 2].set_title('|p_{ref} - p_{NN}|')

    plt.show()

net_params = '/data/checkpoints/RANS_backstep_90000_alpha0.05.pth'
test(net_params=net_params)
